In [1]:
!pkill -f "Microsoft Excel"

In [3]:
import glob
import xlwings as xw
import pandas as pd

In [4]:
xl_app = xw.App(visible=True)
wb = xl_app.books.open('./dev/리스/리스 데이터정리.xlsx')

In [12]:
data = wb.sheets['현대'].range('A1:T2000').value
df = pd.DataFrame(data[1:], columns=data[0])
df = df.dropna(subset = 'brand_name')
df = df[df['판매여부'] == 'Y']

wb.close()

In [36]:
df.head()

,brand_name,car_name,model_name,model_detial,car_price,판매여부,brand_BNK,car_BNK,model_BNK,brand_MZ,car_MZ,brand_woori,car_woori,brand_im,car_im,brand_lotte,car_lotte,model_lotte,brand_sh,car_sh
0,현대,더 뉴 캐스퍼,2025년형 가솔린 1.0\n(2024.10~),스마트,14600000.0,Y,현대,캐스퍼,더 뉴 캐스퍼 가솔린 1.0 디 에센셜 2025년형,1.0,1.0,None,None,현대,캐스퍼 1.0,1.0,25.0,1.0,1.0,캐스퍼 가솔린 1.0
1,현대,더 뉴 캐스퍼,2025년형 가솔린 1.0\n(2024.10~),디 에센셜,16800000.0,Y,현대,캐스퍼,더 뉴 캐스퍼 가솔린 1.0 디 에센셜 2025년형,1.0,1.0,None,None,현대,캐스퍼 1.0,1.0,25.0,1.0,1.0,캐스퍼 가솔린 1.0
2,현대,더 뉴 캐스퍼,2025년형 가솔린 1.0\n(2024.10~),인스퍼레이션,19800000.0,Y,현대,캐스퍼,더 뉴 캐스퍼 가솔린 1.0 디 에센셜 2025년형,1.0,1.0,None,None,현대,캐스퍼 1.0,1.0,25.0,1.0,1.0,캐스퍼 가솔린 1.0
3,현대,더 뉴 캐스퍼,2025년형 가솔린 1.0 터보\n(2024.10~),스마트 + 캐스퍼 액티브Ⅰ,15550000.0,Y,현대,캐스퍼,더 뉴 캐스퍼 가솔린 1.0 디 에센셜 2025년형,1.0,1.0,None,None,현대,캐스퍼 1.0T,1.0,25.0,4.0,1.0,캐스퍼 가솔린 터보 1.0
4,현대,더 뉴 캐스퍼,2025년형 가솔린 1.0 터보\n(2024.10~),디 에센셜 + 캐스퍼 액티브Ⅰ,17750000.0,Y,현대,캐스퍼,더 뉴 캐스퍼 가솔린 1.0 디 에센셜 2025년형,1.0,1.0,None,None,현대,캐스퍼 1.0T,1.0,25.0,4.0,1.0,캐스퍼 가솔린 터보 1.0


In [46]:
xl_app = xw.App(visible=True)
wb = xl_app.books.open('./dev/리스/견적기/bnk.xlsm')

def brand_idx(sh, x):
    x = x.strip()
    models = sh.range('J7', 'J36').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1

def car_idx(sh,x):
    x = x.strip()
    models = sh.range('AM20', 'AM99').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1

def car_model_idx(sh,x):
    x = x.strip()
    models = sh.range('AO20', 'AO256').value 
    models = [str(x).strip().replace('\xa0', ' ') for x in models]
    for idx, model in enumerate(models):
        if model == x:
            model_code = model[0]
            break
    return idx+1

def convert_params(brand, car, model):
    input_data = {
        "bnk": {
            'brand_name': [0, 'B9', brand],
            'car_name': [0, 'B13', car],
            'model_name': [0, 'B15', model],
            'car_price': [1, 'N13', 19940000],
            'deposit': [1, 'N36', 0],
            'downpayment': [1, 'N36', 0],
            'period': [0, 'B39', 2],
        },
    }
    print(input_data)
    return input_data  # 디버깅을 위해 input_data 출력

xl_app.calculation = 'manual'
xl_app.enable_events = False

sheet_names = ['Sheet1', 'Es1']
sheets = [wb.sheets[i] for i in sheet_names]

input_data = df.head(1).apply(lambda x: convert_params(x['brand_BNK'], x['car_BNK'], x['model_BNK']), axis=1)

for i in input_data['bnk'].items():
    val = i[1][2]
    if i[0] == 'brand_name':
        d = brand_idx(sheets[1], val)
        print(d)
    elif i[0] == 'car_name':
        d = car_idx(sheets[1], val)

    elif i[0] == 'model_name':
        d = car_model_idx(sheets[1], val)
    else:
        d = i[1][2]
        
    sheets[i[1][0]].range(i[1][1]).value = d #'현대' #브랜드 => 맵핑 테이블 변경


xl_app.calculation = 'automatic'
xl_app.enable_events = True

{'bnk': {'brand_name': [0, 'B9', '현대'], 'car_name': [0, 'B13', '캐스퍼'], 'model_name': [0, 'B15', '\xa0더\xa0뉴\xa0캐스퍼\xa0가솔린\xa01.0\xa0디\xa0에센셜 2025년형'], 'car_price': [1, 'N13', 19940000], 'deposit': [1, 'N36', 0], 'downpayment': [1, 'N36', 0], 'period': [0, 'B39', 2]}}


KeyError: 'bnk'

In [22]:
df.iloc[0][['brand_name', 'car_name', 'model_name','model_detial','car_price']]

brand_name                              현대
car_name                           더 뉴 캐스퍼
model_name      2025년형 가솔린 1.0\n(2024.10~)
model_detial                           스마트
car_price                       14600000.0
Name: 0, dtype: object